In [79]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from os import listdir
from os.path import isfile, join

from ipywidgets import interact, widgets

In [80]:
p = Path.home()
cof_path = p/'OneDrive'/'Documentos'/'Python'/'Results'/'CSV'
folders = ['PU/Umidade', 'PUEG4/Umidade', 'PP4', 'GPP11']
cof_files = []
for folder in folders:
    cof_files.extend([cof_path/folder/f for f in listdir(cof_path/folder) if isfile(join(cof_path/folder, f))])
hum_path = p/'OneDrive'/'Documentos'/'Python'/'Results'/'CSV'/'Umidade'/'umidade_combinada.csv'
cof_summary_path = p/'OneDrive'/'Documentos'/'Python'/'Results'/'Outputs'/'Umidade'/'data.csv'
parse_dates = ['Timestamp']

In [81]:
hum_df = pd.read_csv(hum_path,
                     usecols = ['Timestamp','Temperatura','Umidade'],
                     parse_dates = parse_dates,
                     dtype = np.float64
                    )

In [82]:
summary_df = pd.read_csv(cof_summary_path, parse_dates = ['Início'])

In [83]:
starts = summary_df.sort_values(by = 'Início').reset_index(drop = True)['Início']
corr_time_diffs = pd.Timedelta(value = 50, unit = 'min') + pd.Timedelta(value = 50, unit = 'sec')

In [84]:
frames = {}
for i, file in enumerate(cof_files):
    cof_df = pd.read_csv(cof_files[0],
                     header=17,
                     skiprows = [21],
                     dtype = {'T': np.float64,'COF': np.float64},
                     usecols = ['T','COF']
                    )
    cof_df['Timedeltas'] = pd.Timedelta(value = 1, unit = 'ms') * cof_df.index
    cof_df['Timestamp'] = starts[i] + cof_df['Timedeltas'] - corr_time_diffs
    if i < 10:
        cof_df['Timestamp'] = starts[i] + cof_df['Timedeltas'] - corr_time_diffs - pd.Timedelta(value = 12, unit = 'min')
    cof_df = cof_df.drop(columns = 'Timedeltas')
    frames[str(file)] = cof_df

In [85]:
interact(lambda key: frames[key], key = list(frames.keys()))

interactive(children=(Dropdown(description='key', options=('C:\\Users\\Caio\\OneDrive\\Documentos\\Python\\Res…

<function __main__.<lambda>(key)>

In [86]:
hum_df['Time_round'] = hum_df['Timestamp'].round('S')
for key, frame in frames.items():
    frame['Time_round'] = frame['Timestamp'].round('S')

In [87]:
for key, frame in frames.items():
    frame = frame.set_index('Time_round').join(hum_df[['Time_round','Umidade','Temperatura']].set_index('Time_round'))
    frames[key] = frame.reset_index(drop = True)

In [88]:
interact(lambda key: frames[key], key = list(frames.keys()))

interactive(children=(Dropdown(description='key', options=('C:\\Users\\Caio\\OneDrive\\Documentos\\Python\\Res…

<function __main__.<lambda>(key)>

In [112]:
def timeline_plot(key):
    df = frames[key].sample(frac = 0.001).sort_values(by = 'Timestamp')
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(
        x = df['Timestamp'], 
        y = df['COF'],
        mode = 'lines',
        name = 'COF',
        marker_color = 'darkgreen',
        legendgroup = 'COF',
         ),
        secondary_y = True,)
    fig.add_trace(go.Scatter(   
        x = df['Timestamp'],           
        y = df['Umidade'],            
        mode = 'lines',
        name = 'Umidade',
        marker_color = 'blue',
        legendgroup = 'Umidade',
         ),
        secondary_y = False,)
    fig.add_trace(go.Scatter(   
        x = df['Timestamp'],           
        y = df['Temperatura'],            
        mode = 'lines',
        name = 'Temperatura',
        marker_color = 'red',
        legendgroup = 'Umidade',
         ),
        secondary_y = False,)
    fig.update_yaxes(title_text = 'COF (-)',
                     title_font_color = 'darkgreen',
                     range = [0,1],
                     secondary_y = True) #only temperature
    fig.update_yaxes(title_text = 'Umidade Relativa (%) / Temperatura (°C)',
                     title_font_color = 'black',
                     range = [0,100],
                     secondary_y = False) #only humidity
    fig.update_layout(template = 'plotly')
    return fig

In [113]:
interact(timeline_plot, key = list(frames.keys()))

interactive(children=(Dropdown(description='key', options=('C:\\Users\\Caio\\OneDrive\\Documentos\\Python\\Res…

<function __main__.timeline_plot(key)>

In [121]:
list(frames.keys())[0][-18:]

'Umidade_FC_001.csv'

In [116]:
paths = {}
for i, key in enumerate(frames.keys()):
    if i < 16:
        path = cof_path/'Umidade'/'PU'
    elif i < 33:
        path = cof_path/'Umidade'/'GP4'
    elif i < 41:
        path = cof_path/'Umidade'/'PP4'
    else:
        path = cof_path/'Umidade'/'GPP11'
    paths[key] = path

In [123]:
for key, frame in frames.items():
    frame.to_csv(paths[key]/key[-18:])